In [6]:
# -- Enhancer Agent --
# This script sets up an agent that enhances queries by extracting structured metadata and filters.
# It uses a set of tools to analyze and transform vague or unstructured queries into clear, structured metadata.
# The agent is designed to work with the LangChain framework and utilizes OpenAI's GPT-4 model.
# The agent is capable of using various tools such as keyword extractors, numeric constraint extractors, and category classifiers.

import os
import sys
from langchain_core.tools import Tool
from langchain.agents import create_react_agent
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import ToolNode
from langgraph.types import Command
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate




In [7]:

import os
import sys

# Go up one level to reach src/
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

print(f"BASE_DIR: {BASE_DIR}")


BASE_DIR: /home/prashant-agrawal/Netflix_Project/src


In [8]:
from tools.enhancer_tools_registry import rag_vector_tool

# Define tools for the enhancer agent
rag_agent_tools = [rag_vector_tool]


# Define tool names for the agent
tool_names = [tool.name for tool in rag_agent_tools]

# Define the tool descriptions
tool_descriptions = [tool.description for tool in rag_agent_tools]

# Build readable tool help text for the prompt
tool_help_text = "\n".join(
    [f"{i+1}. {tool.name} - {tool.description}" for i, tool in enumerate(rag_agent_tools)]
)

In [ ]:



# 🧠 Prompt Template
rag_agent_prompt_template = PromptTemplate.from_template("""
You are a semantic search agent. Your job is to fetch top relevant results from a FAISS vector store using the rag_search_tool.

You have access to the following tools:
{tools}


Use this format:
Question: the input query
Thought: think about what to do
Action: the action to take
Action Input: the tool input
Observation: the result
... repeat if needed
Thought: I now know the search results
Final Answer: the documents retrieved

Begin!

Question: {input}
{agent_scratchpad}
""")

# Format the prompt with tool descriptions and names
formatted_prompt = rag_agent_prompt_template.partial(
    tools=tool_help_text,
    tool_names=", ".join(tool.name for tool in rag_agent_tools)
)

# 🔧 Define the React-style agent
llm = ChatOpenAI(model="gpt-4o")  # Or use your preferred model


# Create the agent
rag_agent = create_react_agent(
    llm=llm,
    tools=rag_vector_tool,
    prompt=formatted_prompt,
    )




NameError: name 'rag_vector_tools' is not defined

In [ ]:
def rag_node(state: dict) -> Command[Literal["supervisor"]]:
    """
    RAG node for retrieving relevant documents using semantic + filter-aware search.
    """
    system_prompt = (
        "You are a vector search agent. Your task is to:\n"
        "- Perform semantic search using the RAG tool.\n"
        "- Use filters if provided in the input.\n"
        "- Return the retrieved documents, nothing more.\n"
        "- Never generate answers, just retrieve."
    )

    messages = [{"role": "system", "content": system_prompt}] + state["messages"]

    response = rag_agent.invoke(messages)

    print("🔍 RAG Node executed. Returning to supervisor.")

    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=response.content,
                    name="rag_agent"
                )
            ]
        },
        goto="supervisor"
    )

In [ ]:
if __name__ == "__main__":
    query = "SaaS startups in Hyderabad funded over ₹50 crore after 2021 offering mobile apps"
    result = rag_agent.invoke({"input": query})
    print(result["output"])